In [1]:
import pandas as pd, numpy as np
import datetime
import openpyxl

from db_utils import write_to_db  
from db_utils import fetch_data 
from utils import *
from data_manipulator import *
from utilities.connections import *

In [2]:
geo_conversions = fetch_data("SELECT id as geo_region_id , name as city FROM geo_regions",connection ="oms")
geo_conversions.loc[len(geo_conversions.index)] = [0,'India'] 
geo_conversions=geo_conversions.sort_values('geo_region_id').reset_index(drop=True)
geo_conversions.head()


Query run time in seconds : 0.8750178813934326


,geo_region_id,city
0,0,India
1,1,Mumbai
2,2,Delhi NCR
3,3,Bangalore
4,4,Hyderabad


In [3]:
q_reactivation='select * from raw_reactivation_business where geo_region_id <= 30 and geo_region_id >= 0 '
connection='redshift'
reactivation_t=fetch_data(q_reactivation,connection)

q_retention='select * from raw_retention_business'
connection='redshift'
retention=fetch_data(q_retention,connection)

q_acquistion = 'select * from raw_acquisition_business'
connection='redshift'
acquistion=fetch_data(q_acquistion,connection)

q_lead = 'select distinct * from raw_leads_business'
connection='redshift'
leads=fetch_data(q_lead,connection)

Query run time in seconds : 9.503783226013184
Query run time in seconds : 0.8559060096740723
Query run time in seconds : 0.8624210357666016
Query run time in seconds : 1.560856819152832


In [4]:
reactivation_t['geo_region_id'].unique()

array([12,  9,  6,  3, 15,  8,  1,  2,  4,  5,  7, 10, 11, 13, 14])

In [5]:
reactivation_t.head()

,month,geo_region_id,acc_customers,acc_g30_l60,acc_g60_l90,acc_g90_l120,acc_g120,month.1,geo_region_id.1,reactivated_customers,...,reactivated_customers_m2,reactivated_l2_m2,reactivated_g3_m2,reactivated_g30_l60_m2,reactivated_g60_l90_m2,reactivated_g90_l120_m2,reactivated_g120_m2,orders_m2,orders_l2_m2,orders_g3_m2
0,2021-06-01,12,452,81,51,34,286,2021-06-01,12,40,...,8.0,3.0,5.0,5.0,1.0,1.0,1.0,23.0,3.0,20.0
1,2021-07-01,9,652,134,209,118,191,2021-07-01,9,59,...,2.0,1.0,1.0,2.0,0.0,0.0,0.0,4.0,1.0,3.0
2,2021-08-01,6,19114,2385,1357,1968,13404,2021-08-01,6,1582,...,636.0,395.0,241.0,417.0,115.0,39.0,65.0,1830.0,547.0,1283.0
3,2021-09-01,3,181195,22166,10239,5849,142941,2021-09-01,3,16604,...,8284.0,4743.0,3541.0,1775.0,3689.0,1318.0,1502.0,26494.0,6542.0,19952.0
4,2021-09-01,15,122,24,13,7,78,2021-09-01,15,13,...,2.0,1.0,1.0,0.0,1.0,1.0,0.0,5.0,2.0,3.0


In [6]:
reactivation_t.month.unique()

array(['2021-06-01', '2021-07-01', '2021-08-01', '2021-09-01',
       '2021-10-01', '2021-11-01', '2021-12-01', '2022-01-01',
       '2022-02-01', '2022-03-01', '2022-07-01', '2022-04-01',
       '2022-05-01', '2022-06-01'], dtype=object)

In [7]:
reactivation_t['month'] = pd.to_datetime(reactivation_t['month']).dt.strftime('%Y-%m-%d')
reactivation=reactivation_t.drop(['month.1','month.2', 'month.3','geo_region_id.1','geo_region_id.2','geo_region_id.3'], axis = 1) 
reactivation=reactivation.drop_duplicates()

retention['month'] = pd.to_datetime(retention['month']).dt.strftime('%Y-%m-%d')
retention=retention.drop_duplicates()

acquistion['month'] = pd.to_datetime(acquistion['month']).dt.strftime('%Y-%m-%d')
acquistion=acquistion.drop_duplicates()

leads['month'] = pd.to_datetime(leads['month']).dt.strftime('%Y-%m-%d')
leads=leads.drop_duplicates()

In [8]:
leads=leads.fillna({'geo_region_id':20.0})

leads=leads.groupby(['month','geo_region_id'])['leads'].sum().reset_index() \
    .assign(attribution='New_Leads').append(leads,ignore_index=False)
leads=movecol(leads, 
             cols_to_move=['month','geo_region_id'], 
             ref_col='attribution',
             place='Before')


In [9]:
total_new_leads=leads.loc[leads['attribution']=='New_Leads']

In [10]:
col_list=['leads']
a=len(total_new_leads)
tot=total_new_leads.head(a)    
unique_geo_regions = [0]+tot['geo_region_id'].unique().tolist()
pan_india_totals= tot.reset_index().groupby(['month'])[col_list].sum()
pan_india_totals['geo_region_id'] = 0
pan_india_totals['attribution'] = 'New_Leads' 
pan_india_totals=pan_india_totals.reset_index()


In [11]:
pan_india_totals.to_csv('pan_india_leads.csv',index=False,header=True)

In [12]:
pan_india_totals_acq=pan_india_totals[['month',	'geo_region_id','attribution','leads']]
total_new_leads.tail()

,leads,month,geo_region_id,attribution
220,2669,2022-07-01,13.0,New_Leads
221,3800,2022-07-01,14.0,New_Leads
222,3223,2022-07-01,15.0,New_Leads
223,163,2022-07-01,16.0,New_Leads
224,5565,2022-07-01,20.0,New_Leads


In [13]:
## Appending the Pan India Leads To Total Leads

total_new_leads=total_new_leads.append(pan_india_totals_acq,ignore_index=True)

In [14]:

#pan_india_leads.to_csv('pan_india_leads.csv',index=False,header=True)
total_new_leads.to_csv('tot_leads_business.csv',index=False,header=True)


In [15]:
col_list_acq=acquistion.columns.values.tolist()
col_list_acq=[n for n in col_list_acq if (n != 'geo_region_id')& (n != 'month') & (n != 'attribution')]
pan_india_totals_acq=get_pan_india_totals_react(acquistion,col_list_acq)

col_list_ret=retention.columns.values.tolist()
col_list_ret= [n for n in col_list_ret if (n != 'geo_region_id')& (n != 'month')]
pan_india_totals_ret=get_pan_india_totals_react(retention,col_list_ret)

col_list_react=reactivation.columns.values.tolist()
col_list_react= [n for n in col_list_react if (n != 'geo_region_id')& (n != 'month')]
pan_india_totals_react=get_pan_india_totals_react(reactivation,col_list_react)


In [16]:
pan_india_totals_acq = movecol(pan_india_totals_acq, 
             cols_to_move=['geo_region_id'], 
             ref_col='month',
             place='After')

pan_india_totals_ret = movecol(pan_india_totals_ret, 
             cols_to_move=['geo_region_id'], 
             ref_col='month',
             place='After')

pan_india_totals_react = movecol(pan_india_totals_react, 
             cols_to_move=['geo_region_id'], 
             ref_col='month',
             place='After')


acquistion=acquistion.append(pan_india_totals_acq,ignore_index=True)
# acquistion=pd.merge(acquistion,total_new_leads,on=['month','geo_region_id'],how='left')
# acquistion=acquistion.fillna({'attribution':'New_Leads'})

retention=retention.append(pan_india_totals_ret,ignore_index=True)
reactivation=reactivation.append(pan_india_totals_react,ignore_index=True)

In [17]:

#Merge The Acquisition DataFrame to Total Leads Data Frame
acquistion=pd.merge(acquistion,total_new_leads,how='left',on=['month','geo_region_id'])


In [18]:
acquistion=acquistion.drop_duplicates()
acquistion.to_csv('leads.csv',index=False,header=True)

In [19]:
acquistion=get_lag_columns(acquistion)
retention=get_lag_columns_ret(retention)
reactivation=get_lag_columns_react(reactivation)

In [20]:
acquistion.sort_values(by=['geo_region_id','month'], inplace=True)
acquistion=get_acquistion_data(acquistion)
acquistion=acquistion[['month',	'geo_region_id',	'customers',	'customer_less_2',	'customer_greater_3',	'orders',	'orders_less_2',	'orders_greater_3',	'customers_m_1',	'customer_less_2_m_1',	'customer_greater_3_m_1',	'orders_m_1',	'orders_less_2_m_1',	'orders_greater_3_m_1',	'customers_m_2',	'attribution',	'leads','Conversion %','order_per_customer','Order/Customer (>=3)','Zero order cust','Orders/Customer_m1','% Retention','% Customer>=3/Active']]

retention.sort_values(by=['geo_region_id','month'], inplace=True)
retention=get_retention_data(retention)

reactivation.sort_values(by=['geo_region_id','month'], inplace=True)
reactivation=get_reactivation_data(reactivation)


In [21]:
reactivation.columns

Index(['month', 'geo_region_id', 'acc_customers', 'acc_g30_l60', 'acc_g60_l90',
       'acc_g90_l120', 'acc_g120', 'reactivated_customers', 'reactivated_l2',
       'reactivated_g3', 'reactivated_g30_l60', 'reactivated_g60_l90',
       'reactivated_g90_l120', 'reactivated_g120', 'orders', 'orders_l2',
       'orders_g3', 'reactivated_customers_m1', 'reactivated_l2_m1',
       'reactivated_g3_m1', 'reactivated_g30_l60_m1', 'reactivated_g60_l90_m1',
       'reactivated_g90_l120_m1', 'reactivated_g120_m1', 'orders_m1',
       'orders_l2_m1', 'orders_g3_m1', 'reactivated_customers_m2',
       'reactivated_l2_m2', 'reactivated_g3_m2', 'reactivated_g30_l60_m2',
       'reactivated_g60_l90_m2', 'reactivated_g90_l120_m2',
       'reactivated_g120_m2', 'orders_m2', 'orders_l2_m2', 'orders_g3_m2',
       'orders_lag', 'reactivated_customers_lag', 'Orders/Customer',
       '%Customer>=3/Total Reactivated', 'percentage reactivation',
       'zero_order_customer', '% Retention', 'Orders/Customer_m1

In [22]:
## Acquisition

selected_columns = ['geo_region_id','month','Conversion %','leads','customers','customer_less_2','customer_greater_3','orders','orders_less_2',
                    'orders_greater_3','order_per_customer','Order/Customer (>=3)']
first_df_dict_acq=get_data_subset_acq(selected_columns,acquistion)

selected_columns =['geo_region_id','month','customers_m_1','customer_less_2_m_1','customer_greater_3_m_1','customers_m_2','Zero order cust','orders_m_1',
                    'orders_less_2_m_1','orders_greater_3_m_1','Orders/Customer_m1']
second_df_dict_acq=get_data_subset_acq(selected_columns,acquistion)

selected_columns =['geo_region_id','month','% Retention','% Customer>=3/Active']
third_df_dict_acq=get_data_subset_acq(selected_columns,acquistion)

## Reactivation
selected_columns =['geo_region_id','month','acc_customers','acc_g30_l60','acc_g60_l90','acc_g90_l120','acc_g120']
first_df_dict_reactivation = get_data_subset_react(selected_columns, reactivation)

selected_columns = ['geo_region_id','month','reactivated_customers','reactivated_l2','reactivated_g3','orders',
                        'orders_l2','orders_g3','Orders/Customer']
second_df_dict_reactivation = get_data_subset_react(selected_columns, reactivation)
    
selected_columns = ['geo_region_id','month','percentage reactivation','%Customer>=3/Total Reactivated','% Retention','reactivated_g30_l60','reactivated_g60_l90','reactivated_g90_l120','reactivated_g120']
third_df_dict_reactivation = get_data_subset_react(selected_columns, reactivation)
    
selected_columns = ['geo_region_id','month','reactivated_customers_m1','reactivated_l2_m1','reactivated_g3_m1',
                        'reactivated_customers_m2','zero_order_customer']
fourth_df_dict_reactivation = get_data_subset_react(selected_columns, reactivation)
    
selected_columns =['geo_region_id','month','orders_m1','orders_l2_m1','orders_g3_m1','reactivated_g30_l60_m1','reactivated_g60_l90_m1',
                       'reactivated_g90_l120_m1']
fifth_df_dict_reactivation = get_data_subset_react(selected_columns, reactivation)


## Retention 
selected_columns = ['geo_region_id','month','customers_retained','less_than_two_order_customers','greater_than_two_order_customers',]
first_df_dict = get_data_subset(selected_columns, retention)
    
selected_columns = ['geo_region_id','month','total_orders','orders_less_than_2','orders_greater_than_2','order_per_customer']
second_df_dict = get_data_subset(selected_columns, retention)

selected_columns = ['geo_region_id','month','m1_customers_retained','m1_less_than_two_order_customers','m1_greater_than_two_order_customers',
                    'm1_retention_history','zero_order_customer','M-1_order_per_customer','%_Retention']
third_df_dict = get_data_subset(selected_columns, retention)



In [23]:
[0]+sorted(reactivation_t['geo_region_id'].unique().tolist())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [24]:
df_dict = {}
#for geo_region in [0,1,2,3,4,5,6,8,10]:
for geo_region in [0]+sorted(reactivation_t['geo_region_id'].unique().tolist()):
    df_dict[str(geo_region)] = {
        'Acquisition Customers & Orders':acquisition_first_metric(first_df_dict_acq[geo_region]),
        'Retention for Acquisition Base':acquisition_second_metric(second_df_dict_acq[geo_region]),
        'Retention % for Acquistion Base':acquisition_third_metric(third_df_dict_acq[geo_region]),
        'Retained Customer Base':retention_first_metric(first_df_dict[geo_region]),
        'Retained Customers Orders':retention_second_metric(second_df_dict[geo_region]),
        'M-1 Retention' :retention_last_month_metric(third_df_dict[geo_region]),
        'Accumulated Base':reactivation_first_metric(first_df_dict_reactivation[geo_region]),
        'Reactivated Customers':reactivation_customer_metric(second_df_dict_reactivation[geo_region]),
        'Reactivation Base':reactivation_days_metric(third_df_dict_reactivation[geo_region]),
        'Retention for Reactivation Base':reactivation_second_metric(fourth_df_dict_reactivation[geo_region]),
        'NextMonth Retention for Reactivation Base':reactivation_third_metric(fifth_df_dict_reactivation[geo_region])
        
        
    }        

In [25]:
# for key in df_dict['7'].keys():
#     df_dict['7'][key].fillna(0, inplace=True)

In [25]:
df_dict['11']['Acquisition Customers & Orders']

month,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-06-01,2022-07-01
Conversion %,5.26,5.81,9.16,10.63,15.61,16.60,15.50,15.92,18.14,22.38,25.09,15.10
New_Leads,38.00,3509.00,3646.00,3734.00,4445.00,3561.00,4311.00,4297.00,4327.00,3678.00,3758.00,5621.00
Customer,2.00,204.00,334.00,397.00,694.00,591.00,668.00,684.00,785.00,823.00,943.00,849.00
Customers <=2,1.00,190.00,297.00,340.00,560.00,464.00,532.00,616.00,702.00,737.00,842.00,759.00
Customers >=3,1.00,14.00,37.00,57.00,134.00,127.00,136.00,68.00,83.00,86.00,101.00,90.00
Order,5.00,287.00,476.00,642.00,1287.00,1170.00,1250.00,1039.00,1186.00,1254.00,1429.00,1315.00
Order <=2,1.00,212.00,333.00,396.00,678.00,575.00,656.00,731.00,837.00,871.00,974.00,890.00
Order >=3,4.00,75.00,143.00,246.00,609.00,595.00,594.00,308.00,349.00,383.00,455.00,425.00
order_per_customer,2.50,1.41,1.43,1.62,1.85,1.98,1.87,1.52,1.51,1.52,1.52,1.55
Order/Customer (>=3),4.00,5.36,3.86,4.32,4.54,4.69,4.37,4.53,4.20,4.45,4.50,4.72


In [26]:
for key in df_dict['7'].keys():
    df_dict['7'][key]= df_dict['7'][key][list( filter(lambda x: x>= '2021-06-01', df_dict['7']['Acquisition Customers & Orders'].columns))]

In [27]:
for key in df_dict['11'].keys():
    df_dict['11'][key]= df_dict['11'][key][list( filter(lambda x: x>= '2021-06-01', df_dict['11']['Acquisition Customers & Orders'].columns))] 


In [28]:
for key in df_dict['12'].keys():
    df_dict['12'][key]= df_dict['12'][key][list( filter(lambda x: x>= '2021-08-01', df_dict['12']['Acquisition Customers & Orders'].columns))]

for key in df_dict['13'].keys():
    df_dict['13'][key]= df_dict['13'][key][list( filter(lambda x: x>= '2021-09-01', df_dict['13']['Acquisition Customers & Orders'].columns))]

for key in df_dict['14'].keys():
    df_dict['14'][key]= df_dict['14'][key][list( filter(lambda x: x>= '2021-10-01', df_dict['14']['Acquisition Customers & Orders'].columns))]
    
for key in df_dict['15'].keys():
    df_dict['15'][key]= df_dict['15'][key][list( filter(lambda x: x>= '2021-12-01', df_dict['15']['Acquisition Customers & Orders'].columns))]
    

In [30]:
    def generate_excel_workbook():
        """ Creates an excel sheet that has necessary information """
        #geo_regions = [0,1,2,3,4,5,6,8,10]
        geo_regions = [0]+sorted(reactivation_t['geo_region_id'].unique().tolist())
        data_frame_dicts = {}
        for geo_region in geo_regions:
            dataframe_array = []
            for table in df_dict[str(geo_region)]:
                write_to_excel = df_dict[str(geo_region)][table].reset_index()
                write_to_excel.rename(columns = {'index' :table}, inplace = True)
                dataframe_array.append(write_to_excel)
            sheet_name = "geo_region_id_" + str(geo_region)
            data_frame_dicts[sheet_name] = dataframe_array
        run_time = (str(datetime.datetime.now()).split(" ")[1].split(".")[0]).replace(":","_")
        sheet_name = "./demand_report_workbook_personal" + ".xlsx" 
        writer = pd.ExcelWriter(sheet_name,engine ='xlsxwriter') 
        workbook  = writer.book
        bold = workbook.add_format({'bold': True})
#         for key in data_frame_dicts.keys():
#             row = 0
#             worksheet.write(row, col, key)
#             row += 1 


        for geo_region, data in data_frame_dicts.items():
            print(data)
            col = 1
            row = 1
            for df in data :
                print("hello")
                add_col = len(df.columns)
                add_row = len(df.index)
                df.to_excel(writer, sheet_name = str(geo_region), 
                     startrow = row, startcol = col, index = False)
                worksheet = writer.sheets[geo_region]
                header_format = workbook.add_format({
                              'bold': True,
                              'text_wrap': True,
                              'valign': 'top',
                              'fg_color': '#003366',
                              'font_color': 'white',
                              'border': 1,
                              'num_format': 'mmm-yy' })
               
                for f, b in zip(df.columns.values, list(range(col, col + add_col))):
                    worksheet.write(row, b , f, header_format)
                row = row + add_row + 4
        writer.save()



In [31]:
generate_excel_workbook()

[month Acquisition Customers & Orders  2021-06-01  2021-07-01  2021-08-01  \
0                       Conversion %       27.40       25.36       26.55   
1                          New_Leads   121139.00   174233.00   193927.00   
2                           Customer    33195.00    44179.00    51496.00   
3                      Customers <=2    28670.00    37864.00    44284.00   
4                      Customers >=3     4525.00     6315.00     7212.00   
5                              Order    58002.00    78276.00    89870.00   
6                          Order <=2    34504.00    45243.00    52927.00   
7                          Order >=3    23498.00    33033.00    36943.00   
8                 order_per_customer        1.75        1.77        1.75   
9               Order/Customer (>=3)        5.19        5.23        5.12   

month  2021-09-01  2021-10-01  2021-11-01  2021-12-01  2022-01-01  2022-02-01  \
0           27.01       26.90       28.33       28.64       25.51       28.50   


5           711.0      1026.0      1064.0      1247.0       979.0       912.0  ]
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
[month Acquisition Customers & Orders  2021-06-01  2021-07-01  2021-08-01  \
0                       Conversion %       25.93       25.35       29.17   
1                          New_Leads    13511.00    18555.00    20874.00   
2                           Customer     3503.00     4703.00     6089.00   
3                      Customers <=2     3120.00     4183.00     5358.00   
4                      Customers >=3      383.00      520.00      731.00   
5                              Order     5479.00     7494.00     9809.00   
6                          Order <=2     3680.00     4880.00     6280.00   
7                          Order >=3     1799.00     2614.00     3529.00   
8                 order_per_customer        1.56        1.59        1.61   
9               Order/Customer (>=3)        4.70        5.03        4.83   

month  2021-09-

[month Acquisition Customers & Orders  2021-06-01  2021-07-01  2021-08-01  \
0                       Conversion %       28.97       22.77       23.30   
1                          New_Leads     4954.00     8846.00     9410.00   
2                           Customer     1435.00     2014.00     2193.00   
3                      Customers <=2     1252.00     1742.00     1885.00   
4                      Customers >=3      183.00      272.00      308.00   
5                              Order     2455.00     3397.00     3756.00   
6                          Order <=2     1512.00     2082.00     2278.00   
7                          Order >=3      943.00     1315.00     1478.00   
8                 order_per_customer        1.71        1.69        1.71   
9               Order/Customer (>=3)        5.15        4.83        4.80   

month  2021-09-01  2021-10-01  2021-11-01  2021-12-01  2022-01-01  2022-02-01  \
0           26.01       30.78       28.37       28.57       25.35       22.94   


5             6.0         8.0        11.0        16.0        12.0        15.0  ]
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
[month Acquisition Customers & Orders  2021-06-01  2021-07-01  2021-08-01  \
0                       Conversion %       40.33       21.76       20.62   
1                          New_Leads     1929.00     5561.00     6056.00   
2                           Customer      778.00     1210.00     1249.00   
3                      Customers <=2      685.00     1077.00     1120.00   
4                      Customers >=3       93.00      133.00      129.00   
5                              Order     1212.00     1872.00     1826.00   
6                          Order <=2      811.00     1250.00     1286.00   
7                          Order >=3      401.00      622.00      540.00   
8                 order_per_customer        1.56        1.55        1.46   
9               Order/Customer (>=3)        4.31        4.68        4.19   

month  2021-09-

hello
hello
hello
[month Acquisition Customers & Orders  2021-10-01  2021-11-01  2021-12-01  \
0                       Conversion %       13.07       14.77       11.11   
1                          New_Leads     2587.00     2315.00     3276.00   
2                           Customer      338.00      342.00      364.00   
3                      Customers <=2      234.00      262.00      276.00   
4                      Customers >=3      104.00       80.00       88.00   
5                              Order      760.00      670.00      715.00   
6                          Order <=2      297.00      321.00      350.00   
7                          Order >=3      463.00      349.00      365.00   
8                 order_per_customer        2.25        1.96        1.96   
9               Order/Customer (>=3)        4.45        4.36        4.15   

month  2022-01-01  2022-02-01  2022-03-01  2022-06-01  2022-07-01  
0           14.02       17.87       24.01       19.81       23.57  
1       

## Share the Below Generated Excel Report  - demand_report_workbook.xlsx

In [32]:
### Extra Totals | Formatting & Designing the Sheet(s) & Workbook | Styling the Workbook 
## Formatting & Designing the Workbook

from openpyxl import load_workbook
from openpyxl import Workbook
wb = openpyxl.load_workbook('demand_report_workbook_business.xlsx')
sheet=wb.active
sheets=wb.sheetnames[0:len(wb.sheetnames)]
for sheet in sheets:    
    ##Fixing the Width of the Metric Header Column
    wb[sheet].column_dimensions['B'].width = 36    
    ## Freezing the Window Pane
    wb[sheet].freeze_panes = 'C2'    
    ## Removing the Grid Lines
    wb[sheet].sheet_view.showGridLines = False
    
wb.save('demand_report_workbook_business.xlsx')     

## Getting the Grand Total of Orders for Pan India & All City

from openpyxl.formula.translate import Translator
wb = openpyxl.load_workbook('demand_report_workbook_business.xlsx')
sheet=wb.active
i=sheet.max_row+4
sheets=wb.sheetnames[0:len(wb.sheetnames)]
#sheets=[n for n in sheets if (n != 'Pune')]
for sheet in sheets:
    print(sheet)
    for j in range(3,wb[sheet].max_column+1):
        ws=wb[sheet]
        wb[sheet].cell(row=i,column=2).value='Total Customers'
        ws.cell(row=i,column=j).value=(ws.cell(row=5,column=j).value)+(ws.cell(row=36,column=j).value)+(ws.cell(row=71,column=j).value)
        wb[sheet].cell(row=i+1,column=2).value='Total Orders'
        wb[sheet].cell(row=i+1,column=j).value=(wb[sheet].cell(row=8,column=j).value)+(wb[sheet].cell(row=43,column=j).value)+(wb[sheet].cell(row=74,column=j).value)  
                
wb.save("demand_report_workbook_business.xlsx")  

## Workbook styling
## Border Styling & Beautification the Grand Totals

from openpyxl.styles.borders import Border, Side
from openpyxl import Workbook

dash_border = Border(left=Side(style='mediumDashDot'), 
                     right=Side(style='mediumDashDot'), 
                     top=Side(style='mediumDashDot'), 
                     bottom=Side(style='mediumDashDot'))

from openpyxl.formula.translate import Translator
wb = openpyxl.load_workbook('demand_report_workbook_business.xlsx')
sheet=wb.active
sheets=wb.sheetnames[0:len(wb.sheetnames)]
sheets
for sheet in sheets:
    wb[sheet].cell(row=111,column=2).border = dash_border
    wb[sheet].cell(row=112,column=2).border = dash_border        
    
wb.save("demand_report_workbook_business.xlsx")  

## Coloring & Naming the Sheets to City & Individual Tab 
## Naming the Sheets
from openpyxl import load_workbook
from openpyxl import Workbook
wb=openpyxl.load_workbook('demand_report_workbook_business.xlsx') 

for i in [0]+sorted(reactivation_t['geo_region_id'].unique().tolist()):
    try:
        indx = (geo_conversions[geo_conversions['geo_region_id']==i]).index[0]
        wb['geo_region_id_{a}'.format(a=i)].title = geo_conversions.iloc[indx]['city']
    except Exception:
        pass    


# ws1=wb['geo_region_id_0'].title='India'
# ws2=wb['geo_region_id_1'].title='Mumbai'
# ws3=wb['geo_region_id_2'].title='Delhi'
# ws4=wb['geo_region_id_3'].title='Bangalore'
# ws5=wb['geo_region_id_4'].title='Hyderabad'
# ws6=wb['geo_region_id_5'].title='Chennai'
# ws7=wb['geo_region_id_6'].title='Ahmedabad'
# ws8=wb['geo_region_id_8'].title='Pune'
# ws9=wb['geo_region_id_10'].title='Surat'

wb.save('demand_report_workbook_business.xlsx') 


## India Flag Pattern
color_map={0:'00FF6600',1:'00FFFFFF',2:'0000FF00'}
wb = openpyxl.load_workbook('demand_report_workbook_business.xlsx')
# for sheet in wb.worksheets:

for index,name in enumerate(wb.sheetnames):
    color = color_map[index%3]
    wb[name].sheet_properties.tabColor = color



#     wb['India'].sheet_properties.tabColor = "00FF6600"
#     wb['Mumbai'].sheet_properties.tabColor = "00FFFFFF"
#     wb['Delhi'].sheet_properties.tabColor = "0000FF00"
#     wb['Bangalore'].sheet_properties.tabColor = "00FF6600"
#     wb['Chennai'].sheet_properties.tabColor = "00FFFFFF"
#     wb['Hyderabad'].sheet_properties.tabColor = "0000FF00"
#     wb['Ahmedabad'].sheet_properties.tabColor = "00FF6600"
#     wb['Pune'].sheet_properties.tabColor = "00FFFFFF"
#     wb['Surat'].sheet_properties.tabColor = "0000FF00"

wb.save("demand_report_workbook_business.xlsx")




geo_region_id_0


TypeError: unsupported operand type(s) for +: 'NoneType' and 'NoneType'

In [32]:
sheets

['geo_region_id_0',
 'geo_region_id_1',
 'geo_region_id_2',
 'geo_region_id_3',
 'geo_region_id_4',
 'geo_region_id_5',
 'geo_region_id_6',
 'geo_region_id_7',
 'geo_region_id_8',
 'geo_region_id_9',
 'geo_region_id_10',
 'geo_region_id_11',
 'geo_region_id_12',
 'geo_region_id_13',
 'geo_region_id_14',
 'geo_region_id_15']